In [1]:
import os
from glob import glob
from pymystem3 import Mystem
import pandas as pd
import json

In [3]:
pp = os.getcwd()
PATH_DD = pp.split('group_mlada')[0] + 'group_mlada/'

In [4]:
text_and_bert_path = f'{PATH_DD}/data/train/text_and_bert.parquet'
text_ful = pd.read_parquet(text_and_bert_path, engine='pyarrow')

In [11]:
text_ful.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2252569 entries, 0 to 2252568
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   variantid     int64 
 1   name          object
 2   description   object
 3   name_bert_64  object
dtypes: int64(1), object(3)
memory usage: 68.7+ MB


In [5]:
text_ful.head(10)

,variantid,name,description,name_bert_64
0,47920382,"Игрушка для ванной , сувенир Уточка Дьяволица",Серия уточек бренда FUNNY DUCKS представлена ...,"[-0.24964939057826996, 0.6433379650115967, 0.4..."
1,49801845,Стразы(бусины) клеевые на листе 9*16 см,Стразы(бусины) клеевые на листе 9*16 см,"[-0.6397916078567505, 0.3660058379173279, 0.67..."
2,49853444,"Набор для вышивания Vervaco ""Подушка. Геометри...",Состав набора: канва-страмин Zweigart с нанесе...,"[-0.27170804142951965, 0.3709857165813446, 0.4..."
3,49893028,"Кружево коклюшечное ""Prym"", цвет: серый, 8 мм,...","Кружево коклюшечное ""Prym"" предназначено для о...","[-0.5911799669265747, 0.45404374599456787, 0.4..."
4,49987483,Оригами Настольная игра Фиксики Кодовый замок,"В настольной игре Оригами ""Фиксики. Кодовый за...","[-0.29846125841140747, 0.41105785965919495, 0...."
5,49995691,"Игра Падающая башня Классика, 54 бруска",Что это такое и как в это играть?<br> &laquo;П...,"[-0.39041391015052795, 0.35741981863975525, 0...."
6,50101275,"Глина для моделирования Fimo ""Soft"", цвет: пол...",Мягкая глина на полимерной основе (пластика) F...,"[-0.3273184895515442, 0.42410966753959656, 0.4..."
7,50373843,"Кабошон круглый, акрил, 12 мм, цвет №23 синий ...","Кабошон круглый, акрил, 12 мм, цвет №23 синий ...","[-0.6374273896217346, 0.6470940113067627, 0.53..."
8,51158084,"Belashoff Одеяло 200x220 см, с наполнителем Ше...",Для любителей долговечных натуральных вещей — ...,"[-0.497112900018692, 0.25157395005226135, 0.44..."
9,51252362,"Kamjove Чайник Гунфу, 900 мл",Чайник Гунфу для удобного и быстрого завариван...,"[-0.4568362236022949, 0.5503043532371521, 0.34..."


In [6]:
attributes_path = f'{PATH_DD}/data/train/attributes.parquet'
attributes = pd.read_parquet(attributes_path, engine='pyarrow')

In [7]:
attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2252569 entries, 0 to 2252568
Data columns (total 3 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   variantid                          int64 
 1   categories                         object
 2   characteristic_attributes_mapping  object
dtypes: int64(1), object(2)
memory usage: 51.6+ MB


In [8]:
attributes.head(10)

,variantid,categories,characteristic_attributes_mapping
0,47920382,"{""1"": ""EPG"", ""2"": ""Детские товары"", ""3"": ""Игру...","{""Цвет товара"": [""бежевый"", ""светло-розовый""],..."
1,49801845,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Количество в упаковке, шт"": [""1""], ""Бренд"": ..."
2,49853444,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Бренд"": [""Vervaco""], ""Тип"": [""Набор для выши..."
3,49893028,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""серый""], ""Ширина, см"": [""0.8..."
4,49987483,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""разноцветный""], ""Название цв..."
5,49995691,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Тип"": [""Настольная игра""], ""Страна-изготовит..."
6,50101275,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""зеленый"", ""прозрачный""], ""На..."
7,50373843,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""синий""], ""Количество в упако..."
8,51158084,"{""1"": ""EPG"", ""2"": ""Дом и сад"", ""3"": ""Подушки, ...","{""Артикул"": [""ОШО 4 - 3 Л""], ""Тип"": [""Одеяло""]..."
9,51252362,"{""1"": ""EPG"", ""2"": ""Дом и сад"", ""3"": ""Столовая ...","{""Артикул"": [""991""], ""Комплектация"": [""- чайни..."


In [9]:
print(len(attributes))

2252569


In [13]:
from collections import Counter
stat_len_categories1 = Counter()
stat_len_categories2 = Counter()
stat_len_categories3 = Counter()
stat_len_categories4 = Counter()


In [14]:
for i in range(0, len(attributes)):
    dd = json.loads(attributes.iloc[i]['categories'])
    stat_len_categories1[dd['1']] +=1
    stat_len_categories2[dd['2']] +=1
    stat_len_categories3[dd['3']] +=1
    stat_len_categories4[dd['4']] +=1

In [15]:
stat_len_categories1

Counter({'EPG': 1746039,
         'Одежда и обувь': 399531,
         'Книги и цифровые книги': 106999})

In [18]:
len(stat_len_categories2)

24

In [24]:
c2 = [(key, vv) for key, vv in stat_len_categories2.most_common()]
c2

[('Дом и сад', 214851),
 ('Одежда', 200703),
 ('Обувь', 198828),
 ('Строительство и ремонт', 198109),
 ('Автотовары', 182467),
 ('Мебель', 109966),
 ('Книги', 106999),
 ('Спорт и отдых', 106036),
 ('Хобби и творчество', 105321),
 ('Электроника', 105162),
 ('Галантерея и аксессуары', 97068),
 ('Красота и гигиена', 71960),
 ('Аптека', 71701),
 ('Канцелярские товары', 71032),
 ('Товары для животных', 70448),
 ('Продукты питания', 70270),
 ('Детские товары', 69399),
 ('Бытовая техника', 69197),
 ('Товары для взрослых', 66991),
 ('Бытовая химия', 66051),
 ('Ювелирные изделия', 5),
 ('Антиквариат и коллекционирование', 2),
 ('Автомототехника', 2),
 ('Фермерское хозяйство', 1)]

In [16]:
stat_len_categories2

Counter({'Детские товары': 69399,
         'Хобби и творчество': 105321,
         'Дом и сад': 214851,
         'Строительство и ремонт': 198109,
         'Бытовая химия': 66051,
         'Товары для взрослых': 66991,
         'Канцелярские товары': 71032,
         'Товары для животных': 70448,
         'Обувь': 198828,
         'Красота и гигиена': 71960,
         'Аптека': 71701,
         'Галантерея и аксессуары': 97068,
         'Бытовая техника': 69197,
         'Мебель': 109966,
         'Продукты питания': 70270,
         'Спорт и отдых': 106036,
         'Одежда': 200703,
         'Автотовары': 182467,
         'Электроника': 105162,
         'Книги': 106999,
         'Ювелирные изделия': 5,
         'Антиквариат и коллекционирование': 2,
         'Фермерское хозяйство': 1,
         'Автомототехника': 2})

In [19]:
len(stat_len_categories3)

583

In [26]:
with open(PATH_DD + 'data/stat_categories3.csv', 'w') as f:
    for key, vv in stat_len_categories3.most_common():
        f.write(f'{key};{vv}\n')

In [20]:
len(stat_len_categories4)

3686

In [27]:
with open(PATH_DD + 'data/stat_categories4.csv', 'w') as f:
    for key, vv in stat_len_categories4.most_common():
        f.write(f'{key};{vv}\n')

In [11]:
print(attributes.iloc[2]['variantid'])
print(attributes.iloc[2]['categories'])
print(json.loads(attributes.iloc[2]['categories'])["1"])
print(attributes.iloc[2]['characteristic_attributes_mapping'])

49853444
{"1": "EPG", "2": "Хобби и творчество", "3": "Набор для рукоделия, творчества", "4": "Набор для вышивания"}
EPG
{"Бренд": ["Vervaco"], "Тип": ["Набор для вышивания"], "Страна-изготовитель": ["Бельгия"], "Материалы набора": ["Акрил", "Бумага", "Канва", "Металл", "Хлопок"], "Техника вышивки": ["Простой крест"], "Возрастные ограничения": ["От 7 лет"], "Длина по горизонтали, см": ["40"], "Длина по вертикали, см": ["40"], "Размеры, мм": ["400 х 400"]}


In [29]:
characteristic = Counter()

In [30]:
for i in range(0, len(attributes)):
    dd = json.loads(attributes.iloc[i]['characteristic_attributes_mapping'])
    for key in dd:
        characteristic[key] += 1

In [31]:
characteristic

Counter({'Цвет товара': 1381119,
         'Пол ребенка': 108569,
         'Бренд': 1746063,
         'Тип': 1931112,
         'Страна-изготовитель': 1748652,
         'Развитие навыков': 5324,
         'Код ТРУ': 6357,
         'Минимальный возраст ребенка': 99692,
         'Материал': 769665,
         'Вес товара, г': 540579,
         'Максимальный возраст ребенка': 54792,
         'Количество в упаковке, шт': 114099,
         'Материалы набора': 13579,
         'Техника вышивки': 6242,
         'Возрастные ограничения': 54138,
         'Длина по горизонтали, см': 17175,
         'Длина по вертикали, см': 17223,
         'Размеры, мм': 235700,
         'Ширина, см': 233423,
         'Состав ниток': 9579,
         'Длина, м': 53577,
         'Название цвета': 732048,
         'Минимальная продолжительность партии, мин.': 1691,
         'Минимальное число игроков': 6270,
         'Максимальное число игроков': 4993,
         'Вид выпуска товара': 316720,
         'Целевая аудитория': 336

In [32]:
with open(PATH_DD + 'data/characteristic.csv', 'w') as f:
    for key, vv in characteristic.most_common():
        f.write(f'{key};{vv}\n')

In [34]:
exploded_df = attributes.explode("categories",ignore_index = True)

In [35]:
exploded_df

,variantid,categories,characteristic_attributes_mapping
0,47920382,"{""1"": ""EPG"", ""2"": ""Детские товары"", ""3"": ""Игру...","{""Цвет товара"": [""бежевый"", ""светло-розовый""],..."
1,49801845,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Количество в упаковке, шт"": [""1""], ""Бренд"": ..."
2,49853444,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Бренд"": [""Vervaco""], ""Тип"": [""Набор для выши..."
3,49893028,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""серый""], ""Ширина, см"": [""0.8..."
4,49987483,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""разноцветный""], ""Название цв..."
...,...,...,...
2252564,1799186009,"{""1"": ""EPG"", ""2"": ""Спорт и отдых"", ""3"": ""Скейт...","{""Материал деки"": [""ABS пластик"", ""Алюминий""],..."
2252565,1801305376,"{""1"": ""Книги и цифровые книги"", ""2"": ""Книги"", ...","{""Год выпуска"": [""2024""], ""Количество страниц""..."
2252566,1802944592,"{""1"": ""EPG"", ""2"": ""Спорт и отдых"", ""3"": ""Аксес...","{""Толщина, мм"": [""0.06""], ""Бренд"": [""Daiwa""], ..."
2252567,1803038155,"{""1"": ""Книги и цифровые книги"", ""2"": ""Книги"", ...","{""Год выпуска"": [""2022""], ""Количество страниц""..."


In [36]:
pd.json_normalize(exploded_df['categories'])

""
0
1
2
3
4
...
2252564
2252565
2252566
2252567


In [6]:
# Получение списка стоп слов
stop_words = []
with open(PATH_DD + 'dataset/stop_word_base.txt', 'r') as f:
    for i in f:
        stop_words.append(i.strip())

In [7]:
def get_dir(NAME_START: str) -> list:
    '''Получение списка директорий наборов
    '''
    dd = [ f.path for f in os.scandir(NAME_START) if f.is_dir() ]
    return dd

In [8]:
def func(value):
    for i in range(10):
        value = value.replace(str(i),'')
    value = value.replace('(-)', ' ')
    return value

In [9]:
def get_array_text_with_category(nf: str, m):
    """ Получение массива текстов по категориям

    Args:
        nf (str): _description_
        m (_type_): _description_
    """
    
    ddel = ['*', ',', '-', '/', '%', ';', ')', '+', '.', '..', ':']
    with open(nf, 'r') as i_f:
        all_text = i_f.read()
        all_text = all_text.strip().rstrip('\r\n')
        all_text = func(all_text)
        #.replace('/\n', ' ')
        for i in ddel:
            all_text = all_text.replace(i, ' ')
        all_text = all_text.lower().split()
    sum_arr = []
    for word in all_text:
        # print(word)
        lemmas = m.lemmatize(word)
        # doc = Doc(word)
        # print(doc.tokens)
        # print(doc.sents)
        # print(lemmas)
        # print()
        if lemmas[0] not in stop_words:
            sum_arr.append(lemmas[0])
        #break
    return sum_arr

In [14]:
NAME_START_POINT = PATH_DD + 'dataset_loc/'
list_dir = get_dir(NAME_START_POINT)
# print(list_dir)
list_dir = ['/home/al/Projects_My/hakaton_ozon_org/dataset_loc/чехлы',
            '/home/al/Projects_My/hakaton_ozon_org/dataset_loc/Наушники',
            '/home/al/Projects_My/hakaton_ozon_org/dataset_loc/платья'
]
            
categories = list_dir.copy()
array_txt = [] # описание как предложения из лемматизированных слов
m = Mystem()
kk = 0
catY = []
keys_cat = []
all_words_list = []
for sd in list_dir:
    key_one = sd.split('/')[-1].lower()
    print(key_one)
    keys_cat.append(key_one)
    list_dir_two = get_dir(sd)
    for dt in list_dir_two:
        list_text = glob(dt + '/*.txt')
        #print(list_text)
        #print()
        servis_list = []
        # servis_list.append(key_one)
        for nf in list_text:
            dd = get_array_text_with_category(nf, m)
            servis_list += dd
            all_words_list += dd
            #break
        array_txt.append(' '.join(servis_list))
        catY.append(kk)
        # print()
        # print(array_txt[kk])
    kk += 1
    #break

# set_dict = []
# for key in array_txt:
#     print()
#     print(key)
#     print(array_txt[key])
#     dd = dict(array_txt[key])
#     nab = []
#     for kk in dd:
#         if dd[kk] >=2 and kk not in stop_words:
#             nab.append(kk)
#     set_dict.append(nab)
set_all_words = set(all_words_list)
print(keys_cat)
print(len(array_txt))
print(catY)
print(len(set_all_words))

чехлы
наушники
платья
['чехлы', 'наушники', 'платья']
32
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
1109


In [15]:
array_txt[2]

'смартфон накладка камера противоударный силиконовый накладка камера kreiger mix специально спроектировать samsung galaxy a g серьезный периметр обладать усиленный угол смартфон средство воздушный подушка находиться между угол накладка серьезно смягчать удар падение толщина предусматривать камера бортик вокруг камера защищать линза отдельность упругий полиуретан безопасный мягкий ощупь накладка иметься отверстие шнурок помощь смартфон можно носить шея рука силиконовый накладка камера противоударный общий изготовитель мес samsung galaxy a g samsung galaxy a смартфон фиолетовый код продавец'

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [17]:
X_train, X_test, y_train, y_test = train_test_split(array_txt, catY, test_size=0.20, random_state=42)
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)

In [18]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [19]:
# Метод наивный Байес
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [20]:
X_test_new = vectorizer.transform(X_test)
#print(X_test_new.shape)
X_new_tfidf = tfidf_transformer.transform(X_test_new)
predicted = clf.predict(X_new_tfidf)

In [21]:
kk = 0
for doc, category in zip(y_test, predicted):
    print(doc, category)
    if doc == category: kk +=1
        
print('model ', kk/len(y_test))

2 2
1 1
2 2
1 1
0 0
0 0
2 2
model  1.0


In [8]:
vectorizer.get_feature_names_out()

array(['00', '1031436306', '11', ..., 'юбка', 'являться', 'яркость'],
      dtype=object)

In [9]:
print(X.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 5 ... 0 2 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]


In [48]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=set_all_words)),
                 ('tfid', TfidfTransformer())]).fit(array_txt)
# tf_transformer = TfidfTransformer(use_idf=False).fit(array_txt)
pipe['count']

CountVectorizer(vocabulary={'a', 'aac', 'acs', 'air', 'amoled', 'android',
                            'apparel', 'apple', 'aramid', 'avakyan', 'awesome',
                            'bass', 'best', 'black', 'bluetooth', 'borofone',
                            'boutique', 'broscorp', 'by', 'c', 'camera', 'case',
                            'cavolo', 'chrome', 'club', 'concept', 'ew',
                            'femme', 'fiber', 'g', ...})

In [49]:
pipe.transform(array_txt).shape

(32, 1109)

In [50]:
pipe['tfid'].idf_

array([4.49650756, 3.1102132 , 3.80336038, ..., 4.49650756, 4.49650756,
       4.49650756])

In [41]:
import sklearn.datasets as ds
list_tem = dir(ds)
len(list_tem)
print(list_tem)

['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__getattr__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_arff_parser', '_base', '_california_housing', '_covtype', '_kddcup99', '_lfw', '_olivetti_faces', '_openml', '_rcv1', '_samples_generator', '_species_distributions', '_svmlight_format_fast', '_svmlight_format_io', '_twenty_newsgroups', 'clear_data_home', 'descr', 'dump_svmlight_file', 'fetch_20newsgroups', 'fetch_20newsgroups_vectorized', 'fetch_california_housing', 'fetch_covtype', 'fetch_kddcup99', 'fetch_lfw_pairs', 'fetch_lfw_people', 'fetch_olivetti_faces', 'fetch_openml', 'fetch_rcv1', 'fetch_species_distributions', 'get_data_home', 'load_breast_cancer', 'load_diabetes', 'load_digits', 'load_files', 'load_iris', 'load_linnerud', 'load_sample_image', 'load_sample_images', 'load_svmlight_file', 'load_svmlight_files', 'load_wine', 'make_biclusters', 'make_blobs', 'make_checkerboard', 'make_circles', 'make_classification', 'make_friedman1'

In [44]:
from sklearn.datasets import load_iris
from sklearn.datasets import __name__
dd = load_iris()
dd.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [46]:
__name__

'sklearn.datasets'

In [9]:
from sklearn.datasets import fetch_20newsgroups

In [10]:
categories = [
    "alt.atheism",
    "talk.religion.misc",
]

# data_train = fetch_20newsgroups(
#     subset="train",
#     categories=categories,
#     shuffle=True,
#     random_state=42,
#     remove=("headers", "footers", "quotes"),
# )
data_all = fetch_20newsgroups()

In [11]:
data_all.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [13]:
data_all.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [14]:
data_all.target

array([7, 4, 4, ..., 3, 1, 8])

In [19]:
print(data_all.data[8])

From: holmes7000@iscsvax.uni.edu
Subject: WIn 3.0 ICON HELP PLEASE!
Organization: University of Northern Iowa
Lines: 10

I have win 3.0 and downloaded several icons and BMP's but I can't figure out
how to change the "wallpaper" or use the icons.  Any help would be appreciated.


Thanx,

-Brando

PS Please E-mail me




In [18]:
data_train.keys() #['data'][0]

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [19]:
data_train['target']

array([0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [20]:
data_train['target_names']

['alt.atheism', 'talk.religion.misc']

In [ ]:

data_test = fetch_20newsgroups(
    subset="test",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=("headers", "footers", "quotes"),
)

print(f"Loading 20 newsgroups dataset for {len(data_train.target_names)} categories:")
print(data_train.target_names)
print(f"{len(data_train.data)} documents")